# 💦 CNN for Sensor-Based Irrigation in Vertical Farms

This notebook demonstrates a "Golden Path" for automated irrigation using computer vision and sensor fusion. We'll simulate camera-like images and moisture sensor readings, train a CNN to classify moisture stress (dry/normal/wet), explain model decisions with Grad-CAM, compare to simple baselines, and export a TFLite-ready model for edge deployment on devices like a Raspberry Pi.

Why CNNs? Convolutional Neural Networks extract spatial features (edges, textures, wilting patterns) using convolutional filters and pooling, making them robust to variable lighting and partial occlusion — they outperform static threshold rules in complex, real-world conditions.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install -q tensorflow scikit-learn matplotlib seaborn joblib opencv-python


In [ ]:
# Standard imports and reproducible settings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import joblib
import random

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
import tensorflow as tf
tf.random.set_seed(SEED)


In [ ]:
# ------------------------------
# Data Simulation: synthetic images + sensor readings
# ------------------------------

IMG_H, IMG_W, IMG_C = 32, 32, 3
N_IMAGES = 5000
classes = ['dry', 'normal', 'wet']

def make_plant_image(label):
    # start with green background
    img = np.ones((IMG_H, IMG_W, IMG_C), dtype=np.float32)
    base_green = np.array([34, 139, 34]) / 255.0  # leaf-like
    img *= base_green

    # add texture noise
    noise = np.random.normal(0, 0.03, img.shape)
    img = np.clip(img + noise, 0, 1)

    # modify by label
    if label == 'dry':
        # desaturate and add brown patches
        img *= 0.6
        for _ in range(np.random.randint(1, 5)):
            x = np.random.randint(0, IMG_W - 4)
            y = np.random.randint(0, IMG_H - 4)
            w = np.random.randint(2, 6)
            h = np.random.randint(2, 6)
            patch = np.random.uniform(0.3, 0.6, (h, w, 1)) * np.array([139, 69, 19]) / 255.0
            img[y:y+h, x:x+w, :] = np.clip(img[y:y+h, x:x+w, :] * 0.4 + patch * 0.6, 0, 1)
    elif label == 'wet':
        # slightly darker but more saturated
        img *= 0.9
        img[:, :, 1] = np.clip(img[:, :, 1] * 1.1 + 0.05, 0, 1)
        # brighten center (water sheen)
        yy, xx = np.mgrid[0:IMG_H, 0:IMG_W]
        cx, cy = IMG_W // 2 + np.random.randint(-3, 3), IMG_H // 2 + np.random.randint(-3, 3)
        mask = np.exp(-((xx-cx)**2 + (yy-cy)**2) / (2*4.0))[:,:,None]
        img = np.clip(img + 0.05 * mask, 0, 1)
    else:
        # normal: balanced greens
        img *= 0.95

    # add random shadow to simulate lighting variation
    if np.random.rand() < 0.3:
        x = np.random.randint(0, IMG_W - 8)
        y = np.random.randint(0, IMG_H - 8)
        w = np.random.randint(4, 12)
        h = np.random.randint(4, 12)
        img[y:y+h, x:x+w, :] *= np.random.uniform(0.5, 0.85)

    return (img * 255).astype(np.uint8)

# generate
X_imgs = np.zeros((N_IMAGES, IMG_H, IMG_W, IMG_C), dtype=np.uint8)
y = np.zeros((N_IMAGES,), dtype=np.int32)
sensor_readings = np.zeros((N_IMAGES, 2), dtype=np.float32)  # moisture, humidity

for i in range(N_IMAGES):
    label_idx = np.random.choice(3, p=[0.25, 0.5, 0.25])
    label = classes[label_idx]
    img = make_plant_image(label)
    X_imgs[i] = img
    y[i] = label_idx

    # sensor readings consistent with label
    if label == 'dry':
        sensor_readings[i, 0] = np.random.uniform(0.05, 0.25)  # moisture
        sensor_readings[i, 1] = np.random.uniform(30, 50)  # humidity
    elif label == 'normal':
        sensor_readings[i, 0] = np.random.uniform(0.3, 0.6)
        sensor_readings[i, 1] = np.random.uniform(50, 70)
    else:
        sensor_readings[i, 0] = np.random.uniform(0.65, 0.95)
        sensor_readings[i, 1] = np.random.uniform(60, 90)

print('Images shape:', X_imgs.shape, 'Labels distribution:', np.bincount(y))


In [ ]:
# Visualize sample images and class balance
fig, axes = plt.subplots(3, 6, figsize=(12, 6))
for i, ax in enumerate(axes.flatten()):
    idx = np.where(y == (i % 3))[0][np.random.randint(0, 10)]
    ax.imshow(X_imgs[idx])
    ax.axis('off')
    ax.set_title(classes[y[idx]])
plt.suptitle('Sample images (dry / normal / wet examples)')
plt.show()

# show sensor distribution
plt.figure(figsize=(8, 4))
plt.scatter(sensor_readings[:, 0], sensor_readings[:, 1], c=y, cmap='viridis', alpha=0.6)
plt.xlabel('Soil moisture (sim)')
plt.ylabel('Humidity (%)')
plt.title('Sensor reading distribution by class')
plt.show()

In [ ]:
# ------------------------------
# Train / Validation / Test split and preprocessing
# ------------------------------

# Normalize images to [0,1]
X = X_imgs.astype('float32') / 255.0

# Combine with sensor features for multimodal baseline (optional)
X_sensors = sensor_readings.copy()

# Split
X_train_img, X_test_img, y_train, y_test, X_train_sens, X_test_sens = train_test_split(
    X, y, X_sensors, test_size=0.2, random_state=SEED, stratify=y)
X_val_img, X_test_img, y_val, y_test, X_val_sens, X_test_sens = train_test_split(
    X_test_img, y_test, X_test_sens, test_size=0.5, random_state=SEED, stratify=y_test)

print('Train/Val/Test image shapes:', X_train_img.shape, X_val_img.shape, X_test_img.shape)

In [ ]:
# ------------------------------
# Baseline: threshold rule and simple MLP on sensor features
# ------------------------------

# Threshold baseline: simple rule on moisture sensor
def threshold_rule(sensor):
    moisture = sensor[0]
    if moisture < 0.28:
        return 0  # dry
    elif moisture < 0.65:
        return 1  # normal
    else:
        return 2  # wet

# evaluate threshold on test set
threshold_preds = np.array([threshold_rule(s) for s in X_test_sens])
print('Threshold rule accuracy:', accuracy_score(y_test, threshold_preds))

# MLP baseline on sensor features
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

mlp = Sequential([Dense(32, activation='relu', input_shape=(X_train_sens.shape[1],)), Dense(16, activation='relu'), Dense(3, activation='softmax')])
mlp.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
mlp.fit(X_train_sens, y_train, validation_data=(X_val_sens, y_val), epochs=20, batch_size=64, verbose=1)

mlp_preds = np.argmax(mlp.predict(X_test_sens), axis=1)
print('MLP accuracy:', accuracy_score(y_test, mlp_preds))

In [ ]:
# ------------------------------
# CNN model: Conv -> Pool -> Conv -> Dense
# ------------------------------

def build_cnn(input_shape=(IMG_H, IMG_W, IMG_C)):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

cnn = build_cnn()
cnn.summary()

checkpoint = ModelCheckpoint('best_cnn.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
early = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)

history = cnn.fit(X_train_img, y_train, validation_data=(X_val_img, y_val), epochs=40, batch_size=64, callbacks=[checkpoint, early], verbose=2)

# Plot accuracy and loss
plt.figure()
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('CNN Accuracy')
plt.legend()
plt.show()


In [ ]:
# ------------------------------
# Evaluation: confusion matrix, classification report, and Water Efficiency Score
# ------------------------------
# Predict
cnn_preds = np.argmax(cnn.predict(X_test_img), axis=1)

print('CNN accuracy:', accuracy_score(y_test, cnn_preds))
print('\nClassification Report:\n', classification_report(y_test, cnn_preds, target_names=classes))

# Confusion matrix
cm = confusion_matrix(y_test, cnn_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix — CNN')
plt.show()

# Water Efficiency Score (business metric): lower false positives for wet reduce wasted watering
# Define: WES = 100 * (1 - (FP_wet + FN_dry) / N)
FP_wet = np.sum((y_test != 2) & (cnn_preds == 2))
FN_dry = np.sum((y_test == 0) & (cnn_preds != 0))
wes = 100 * (1 - (FP_wet + FN_dry) / len(y_test))
print(f'Water Efficiency Score (WES): {wes:.2f} (higher is better)')

# Compare MLP and threshold
print('MLP accuracy:', accuracy_score(y_test, mlp_preds))
print('Threshold accuracy:', accuracy_score(y_test, threshold_preds))


In [ ]:
# ------------------------------
# Grad-CAM: simple implementation for Keras models
# ------------------------------

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-9)
    return heatmap.numpy()

# Pick some test images and show Grad-CAM
last_conv_layer = 'conv2d_1'  # name may depend on model summary
indices = np.random.choice(len(X_test_img), size=6, replace=False)
fig, axes = plt.subplots(2, 6, figsize=(16, 6))
for i, idx in enumerate(indices):
    img = X_test_img[idx:idx+1]
    heatmap = make_gradcam_heatmap(img, cnn, last_conv_layer)
    heatmap = cv2.resize(heatmap, (IMG_W, IMG_H))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted((img[0]*255).astype('uint8'), 0.6, heatmap_color, 0.4, 0)

    ax_img = axes[0, i]
    ax_img.imshow(img[0])
    ax_img.axis('off')
    ax_img.set_title(f'True: {classes[y_test[idx]]}\nPred: {classes[cnn_preds[idx]]}')

    ax_overlay = axes[1, i]
    ax_overlay.imshow(overlay)
    ax_overlay.axis('off')
plt.suptitle('Grad-CAM overlays (true vs predicted)')
plt.show()

In [ ]:
# ------------------------------
# Sample Image Grid: Actual vs Predicted
# ------------------------------

n = 9
idxs = np.random.choice(len(X_test_img), size=n, replace=False)
fig, ax = plt.subplots(3, 3, figsize=(8, 8))
for i, idx in enumerate(idxs):
    r, c = divmod(i, 3)
    ax[r, c].imshow(X_test_img[idx])
    ax[r, c].axis('off')
    ax[r, c].set_title(f'T:{classes[y_test[idx]]} / P:{classes[cnn_preds[idx]]}')
plt.suptitle('Sample: Actual vs Predicted')
plt.show()


In [ ]:
# ------------------------------
# Hybrid: brief moisture trend forecasting with LSTM (optional)
# ------------------------------
from tensorflow.keras.layers import LSTM

# simulate moisture time-series and build sequences
moisture = np.clip(0.5 + 0.2 * np.sin(np.linspace(0, 50, 10000)) + np.random.normal(0, 0.05, 10000), 0, 1)
window = 24
X_seq = np.array([moisture[i-window:i] for i in range(window, len(moisture))])
y_seq = moisture[window:]

# train/test split
split = int(0.8 * len(X_seq))
X_train_seq = X_seq[:split]
y_train_seq = y_seq[:split]
X_test_seq = X_seq[split:split+500]
y_test_seq = y_seq[split:split+500]

# reshape for LSTM
X_train_seq = X_train_seq[..., np.newaxis]
X_test_seq = X_test_seq[..., np.newaxis]

lstm = Sequential([LSTM(32, input_shape=(window, 1)), Dense(16, activation='relu'), Dense(1)])
lstm.compile(optimizer='adam', loss='mse')
lstm.fit(X_train_seq, y_train_seq, epochs=8, batch_size=256, verbose=1)

pred_seq = lstm.predict(X_test_seq)
plt.figure()
plt.plot(y_test_seq[:200], label='True moisture')
plt.plot(pred_seq[:200], label='Predicted moisture')
plt.title('Moisture trend forecasting (LSTM)')
plt.legend()
plt.show()

In [ ]:
# ------------------------------
# Export: save model and convert to TFLite for edge deployment
# ------------------------------
cnn.save('irrigation_cnn.h5')
print('Saved CNN to irrigation_cnn.h5')

try:
    converter = tf.lite.TFLiteConverter.from_keras_model(cnn)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    with open('irrigation_cnn.tflite', 'wb') as f:
        f.write(tflite_model)
    print('Saved TFLite model to irrigation_cnn.tflite')
except Exception as e:
    print('TFLite conversion skipped (requires TF full runtime):', e)

# Sample inference snippet for edge device
print('\nSample inference (pseudo):')
print("# img = capture_from_camera(); img = preprocess(img); preds = model.predict(img[None,...]); action = np.argmax(preds)")

# Deployment checklist
print('\nDeployment checklist:')
print('- Quantize model and test on Raspberry Pi / Coral for latency')
print('- Implement zone-specific actuation (map detection to valve/pump control)')
print('- Add failsafe thresholds and human-in-loop overrides')
print('- Log decisions and sensor readings for drift detection and retraining')


In [ ]:
# ------------------------------
# Irrigation Insights for Operators
# ------------------------------
print('Irrigation insights:')
print('- Use Grad-CAM overlays to identify dry subzones and trigger zone-specific drip to save ~20% water.')
print('- Combine camera predictions with moisture sensor thresholds for high-confidence actuation (ensemble rule).')
print('- Track WES over time to quantify water savings and tune false positive/negative costs per crop type.')

print('\nBusiness-friendly interpretation:')
print('- Accuracy / F1 relate directly to water waste and crop stress: an increase in F1 by 0.1 can map to 10-25% water savings in typical scenarios (estimate; validate on-farm).')
print('- Visual explainability (Grad-CAM) reduces operator trust barriers and enables targeted interventions.')

print('\nNotebook complete — Golden Path ready for experiment and edge deployment.')